# Q1

```
curl localhost:9200
```

{
  "name" : "",
  "cluster_name" : "",
  "cluster_uuid" : "",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "",
    "build_date" : "",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "0",
    "minimum_index_compatibility_version" : "0"
  },

# Q2

In [5]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [25]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questionss"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questionss'})

In [28]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

# Q3

In [29]:
query = 'How do I execute a command in a running docker container?'

def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    result_scores = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        result_scores.append(hit['_score'])
    
    return (result_docs, result_scores)

In [30]:
search_result= elastic_search(query)

In [31]:
search_result

([{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'},
  {'text': 'In case runnin

# Q4

In [32]:
query = 'How do I execute a command in a running docker container?'

def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])    
    return (result_docs)

In [33]:
search_result= elastic_search(query)
search_result

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from

In [34]:
for res in search_result:
    print(res['question'])

How do I debug a docker container?
How do I debug a docker container?
How do I copy files from my local machine to docker container?


# Q5

In [35]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [36]:
prompt = build_prompt(query, search_result)

In [37]:
len(prompt)

1399

# Q6

In [21]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.8 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 17.2 MB/s eta 0:00:00m eta 0:00:01


In [38]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [39]:
tokens=encoding.encode(prompt)

In [40]:
len(tokens)

310

In [41]:
encoding.decode_single_token_bytes(63842)

b"You're"

# Q7 Optional

In [42]:
import os
from groq import Groq

# Instantiation of Groq Client
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY")
)

In [43]:
def llm(prompt):
    response = client.chat.completions.create(
    messages=[
        {
           "role": "user", "content": prompt
        }
    ],
    model="mixtral-8x7b-32768",
    )
    return response.choices[0].message.content

In [44]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [45]:
rag(query)

"To execute a command in a running Docker container, you can use the `docker exec` command. Here's how to do it:\n\n1. First, find the ID of the running Docker container using the command `docker ps`.\n2. Then, use the `docker exec` command to run the desired command in the specific container. The basic syntax is as follows:\n\ndocker exec -it <container-id> <command>\n\nReplace `<container-id>` with your container's ID and `<command>` with the command you want to execute. For example, to start a bash shell in the running container, use:\n\ndocker exec -it <container-id> bash"

# Q8 Optional

1000 requests:

150000 tokens sent: 1000 tokens $ 0.005 -> 0.005 * 150 = 0.75
250000 tokens received: 1000 tokens $ 0.015 -> 0.015 * 250 = 3.75

Total: 0.75 + 3.75 = $ 4.5
